In [1]:
import inspect

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold

from metric.compute_metric import compute_clear
from utils import make_pred, save_data
from metric.read_file import read_file
import numpy as np


In [2]:
from utils import iter_frame, Corp
from utils.identity_assignation import identity_assignation


class InertieClasificator(BaseEstimator, ClassifierMixin):
    def __init__(self, coef_ine_hd = 1, coef_ine_bd =1, coef_dist_hd = 1, coef_dist_bd = 1, 
                 coef_size_hd = 1, coef_size_bd = 1, coef_pt_cache = 10, mem_length= 20, coef_col = 1):
        
        args, _, _, values = inspect.getargvalues(inspect.currentframe())
        values.pop("self")

        for arg, val in values.items():
            setattr(self, arg, val)
    
    def fit(self, X, y=None):
        # on n'apprend rien pour l'instant
        pass
    
    def mk_dist(self):
        def fdist(id, corp):
            var = id.comp(corp)
            dist = var[0] * self.coef_ine_hd
            dist += var[1] * self.coef_ine_bd
            dist += var[2] * self.coef_col
            dist += var[3] * self.coef_dist_hd
            dist += var[4] * self.coef_dist_bd
            dist += var[5] * self.coef_pt_cache
            return dist
        return fdist
               
    
    def predict(self, X, y=None):
        # prediction
        
        def func (data):
            identites = dict()
            for frame in iter_frame(data):
                nb_personne = frame.shape[0]
        
                identites = identity_assignation(frame, identites, None, self.mk_dist())
    
                for i, l in enumerate(frame):
                    cp = Corp(l)
                    yield (identites[cp], 0)  
        
        return [make_pred(vid, func) for vid in X]
    
    def score(self, X, y, sample_weight=None):
        pred = self.predict(X)
        res = 0
        for vid, raw in zip(pred, y):
            c = compute_clear(raw, vid, 1)
            res += c[0]
        # print(res/len(y))
        return res/len(y)
           
        

In [3]:
PATH = "data/Scenario04-01/"
data_ref = read_file(PATH+"groundTruth.txt")
data = np.loadtxt(PATH+"augmented_data.txt")
est = InertieClasificator()
print(est.score([data], [data_ref]))


0.980935875217


In [5]:
from utils import mult_load

data_ref, data = mult_load(["data/Scenario03-02/", "data/Scenario01-01/", "data/Scenario02-03/", "data/Scenario02-01/",
                            "data/Scenario02-02/", "data/Scenario03-03/", "data/Scenario03-02/", "data/Scenario03-01/",
                            "data/Scenario03-05/"])

In [6]:
est2 = InertieClasificator(coef_pt_cache= 20, coef_dist_bd=5, coef_dist_hd=5, coef_ine_hd= 5,
                          coef_ine_bd=20, coef_size_bd=1, coef_size_hd=0, mem_length=30)
print(est2.score(data, data_ref))

0.979901567304094


In [11]:
from utils import iter_frame, Corp
from utils.identity_assignation import identity_assignation
import pickle


class TrainedClasificator(BaseEstimator, ClassifierMixin):
 
    def __init__(self, estimateur):
        
        with open(estimateur) as file:
            self.est = pickle.load(file)
            
        args, _, _, values = inspect.getargvalues(inspect.currentframe())
        values.pop("self")

        for arg, val in values.items():
            setattr(self, arg, val)
    
    def fit(self, X, y=None):
        # on n'apprend rien pour l'instant
        pass
    
    def mk_dist(self):
        def fdist(id, corp):
            return 1-self.est.predict([id.comp(corp)])[0]
        return fdist
               
    
    def predict(self, X, y=None):
        # prediction
        
        def func (data):
            identites = dict()
            for frame in iter_frame(data):
                identites = identity_assignation(frame, identites, None, self.mk_dist())
                for i, l in enumerate(frame):
                    cp = Corp(l)
                    yield (identites[cp], 0)  
        
        return [make_pred(vid, func) for vid in X]
    
    def score(self, X, y, sample_weight=None):
        pred = self.predict(X)
        res = 0
        for vid, raw in zip(pred, y):
            c = compute_clear(raw, vid, 1)
            res += c[0]
        # print(res/len(y))
        return res/len(y)
           
        

In [15]:
from utils import mult_load

data_ref, data = mult_load(["data/Scenario03-02/", "data/Scenario01-01/", "data/Scenario02-03/", "data/Scenario02-01/",
                            "data/Scenario02-02/", "data/Scenario03-03/", "data/Scenario03-02/", "data/Scenario03-01/",
                            "data/Scenario03-05/"])

In [8]:
est = TrainedClasificator("learn/estimateur_grad_boost_max")
print(est2.score(data, data_ref))

0.979901567304094


In [39]:
save_data(res[0], "tmp")

In [38]:
res = est.predict("data/Scenario03-01/")

In [12]:
PATH = "data/Scenario04-01/"
data_ref = read_file(PATH+"groundTruth.txt")
data = np.loadtxt(PATH+"augmented_data.txt")
est = TrainedClasificator("learn/estimateur_grad_boost_max")
est.score([data], [data_ref])

0.9771682616230879